# First Howemork. Optimization and Analytics

#### Done by Javier Alzuaz & Jaime Lobato

### Motivation for the Optimization problem: improving the Indiana Pacers’ performance through optimal shot allocation and lineup selection

The Indiana Pacers reached the NBA Finals last season, establishing themselves as one of the strongest teams in the league. However, their performance this year has significantly got worse, mainly due to a combination of injuries, inconsistent rotations, and the need to rely more heavily on G-League call-ups to maintain roster depth.

This sudden transition from contending for a championship to being in the last place in the NBA standings creates a realistic decision-making scenario for applying optimization techniques.

The coaching staff faces some important questions:

- Given the current roster situation, how should the team allocate its offensive shots among the available players to maximize expected scoring?

- Given the current roster performances, injuries, and the mix of NBA and G-League players, what starting lineup would maximize the team’s on-court effectiveness?

This is a strategic problem with limited resources (healthy players, budget) and clear performance metrics, making it ideal for formulation as a Linear Optimization and later a Mixed-Integer Optimization model.

## Part a. Linear Optimization Model - Shot Allocation

### Mathematical Formulation of the Problem

### Sets

  - $P$: set of all players available.
  - $T = \{2P, 3P\}$: set of shot types.

Position subsets:
  - $P^{PG} \subseteq P$ represents the set of point guards (PGs).
  - $P^{SG} \subseteq P$ represents the set of shooting guards (SGs).
  - $P^{F} \subseteq P$ represents the set of forwards (SFs or PFs).
  - $P^{C} \subseteq P$ represents the set of centers (Cs).

### Parameters

For each player $i \in P$ and $t \in T$:

- $e_{i, t}$: expected points per shot of type $t$ and player $i$.

For example: $e_{i, 2P} = 2 \cdot 2P\%_{i}, \quad e_{i, 3P} = 3 \cdot 3P\%_{i}$

- $S_{i, 2}^{\max}: \text{ max 2-point shots for player } i$ 
- $S_{i, 3}^{\max}: \text{ max 3-point shots for player } i$


With this, we ensure that the shot selection is distributed along the entire roster.

Team parameter:

- $S^{team}$: total number of shots attempts taken by the team in a game.

### Decision Variables

$$s_{i, t} \geq 0$$

- $s_{i, t}$: number of shots of type $t$ each player $i$ takes.


### Objective Function

We want to maximize the expected total points.

\begin{alignat*}{2}
\max \quad & \sum_{i \in P} \sum_{t \in T} e_{i,t} s_{i,t} \\
\text{s.t.} \quad & \sum_{i \in P} \sum_{t \in T} s_{i,t} = S^{\text{team}} & \quad & \text{(total shots)} \\
& \sum_{t \in T} s_{i,2} \leq S_{i,2}^{\max} & \quad & \forall i \in P \quad \text{(player max 2-point shots)} \\
& \sum_{t \in T} s_{i,3} \leq S_{i,3}^{\max} & \quad & \forall i \in P \quad \text{(player max 3-point shots)} \\
& s_{i,t} \geq 0 & \quad & \forall i \in P, t \in T \quad \text{(non-negativity)}
\end{alignat*}


In [45]:
import pandas as pd
from pyomo.environ import *
import numpy as np

data = pd.read_csv("pacers_stats.csv" ,sep =";", encoding='latin-1')

# We might also need per game stats for making some calculations easier
data_per_game = pd.read_csv("pacers_stats_per_game.csv" ,sep =",", encoding='latin-1')

data_per_game

,Rk,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player-additional
0,1.0,Bennedict Mathurin,23.0,SF,2,2,36.5,8.5,15.5,0.548,...,1.5,5.5,7.0,2.5,0.0,0.0,2.5,3.0,31.0,mathube01
1,2.0,Pascal Siakam,31.0,PF,12,12,35.0,8.8,19.3,0.453,...,1.8,5.3,7.1,5.1,1.2,0.3,2.4,3.3,24.6,siakapa01
2,3.0,Aaron Nesmith,26.0,SF,11,11,30.5,4.9,13.4,0.367,...,1.5,2.9,4.5,1.5,0.8,0.3,0.7,2.5,15.5,nesmiaa01
3,4.0,Jarace Walker,22.0,PF,13,8,29.2,3.3,11.3,0.293,...,0.7,4.7,5.4,3.2,0.5,0.5,2.2,2.1,10.2,walkeja02
4,5.0,Andrew Nembhard,26.0,PG,6,6,28.0,5.5,13.8,0.398,...,0.2,1.7,1.8,6.5,0.3,0.2,2.0,2.5,18.0,nembhan01
5,6.0,Obi Toppin,27.0,PF,3,0,27.3,5.0,12.0,0.417,...,1.0,5.7,6.7,1.7,1.0,0.0,2.0,2.3,14.0,toppiob01
6,7.0,Ben Sheppard,24.0,SG,13,5,25.1,2.4,7.5,0.316,...,0.8,3.7,4.5,1.6,0.5,0.2,0.7,2.6,6.5,sheppbe01
7,8.0,Jeremiah Robinson-Earl,25.0,PF,8,3,20.9,2.1,5.6,0.378,...,2.8,3.6,6.4,0.9,0.6,0.0,0.8,1.1,5.6,robinje02
8,9.0,Quenton Jackson,27.0,PG,5,3,20.2,4.0,7.4,0.541,...,1.0,2.4,3.4,3.6,1.0,0.2,1.2,2.0,11.8,jacksqu01
9,10.0,James Wiseman,24.0,C,1,1,20.0,2.0,3.0,0.667,...,4.0,0.0,4.0,0.0,0.0,1.0,3.0,2.0,4.0,wisemja01


We create parameters for the model

In [46]:
# Index for players
players_idx = list(data_per_game.index)

# Shot types
shot_types = ["2P", "3P"]

# Build expected points per shot: e_{i,2P} = 2 * 2P%, e_{i,3P} = 3 * 3P%
e = {}
for i in players_idx:
    two_p_pct = data_per_game.loc[i, "2P%"]
    three_p_pct = data_per_game.loc[i, "3P%"]
    if pd.isna(three_p_pct):
        three_p_pct = 0.0
    e[(i, "2P")] = 2 * two_p_pct
    e[(i, "3P")] = 3 * three_p_pct


""" Max shots per player: 1.25 * their FGA (from the stats file). We use 1.25 as a scaling factor to allow for some flexibility, 
but we want to limit it based on their usual attempts """

S2_max = {}
S3_max = {}

for i in players_idx:
    two_pa = data_per_game.loc[i, "2PA"]   # average 2-point attempts per game
    three_pa = data_per_game.loc[i, "3PA"] # average 3-point attempts per game (may be NaN)

    # Max 2P = 1.25 * 2PA
    S2_max[i] = 1.25 * two_pa

    # Max 3P = 1.25 * 3PA
    S3_max[i] = 1.25 * three_pa


# Total team shots in a game. According to the stats file, the Pacers attempt around 95 shots per game.
S_team = 95


We build the model

In [47]:
from pyomo.environ import *

from itertools import combinations  

model = ConcreteModel()

# Sets
model.PLAYERS = Set(initialize=players_idx)
model.TYPES = Set(initialize=shot_types)

# Parameters
model.e = Param(model.PLAYERS, model.TYPES, initialize=e)
model.S2_max = Param(model.PLAYERS, initialize=S2_max)
model.S3_max = Param(model.PLAYERS, initialize=S3_max)
model.S_team = Param(initialize=S_team)

# Decision variables: s_{i,t} >= 0
model.s = Var(model.PLAYERS, model.TYPES, domain=NonNegativeReals)


In [48]:
# We define the function to maximize: total expected points

def objective_rule(m):
    return sum(m.e[i, t] * m.s[i, t] for i in m.PLAYERS for t in m.TYPES)

model.OBJ = Objective(rule=objective_rule, sense=maximize)


In [49]:
# We define the constraints

# 1. Total shots constraint
def total_shots_rule(m):
    return sum(m.s[i, t] for i in m.PLAYERS for t in m.TYPES) == m.S_team

model.TotalShots = Constraint(rule=total_shots_rule)

# Max 2P attempts per player
def max_2p_rule(m, i):
    return m.s[i, "2P"] <= m.S2_max[i]

model.Max2PShots = Constraint(model.PLAYERS, rule=max_2p_rule)

# Max 3P attempts per player
def max_3p_rule(m, i):
    return m.s[i, "3P"] <= m.S3_max[i]

model.Max3PShots = Constraint(model.PLAYERS, rule=max_3p_rule)


model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)



In [50]:
# We solve the model

solver = SolverFactory("glpk")  # or "cbc", "gurobi", etc.

result = solver.solve(model, tee=True)

print(result.solver.termination_condition)


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\jaime\AppData\Local\Temp\tmpfceyiy2c.glpk.raw --wglp C:\Users\jaime\AppData\Local\Temp\tmps7vil853.glpk.glp
 --cpxlp C:\Users\jaime\AppData\Local\Temp\tmp3l_kodq5.pyomo.lp
Reading problem data from 'C:\Users\jaime\AppData\Local\Temp\tmp3l_kodq5.pyomo.lp'...
43 rows, 42 columns, 84 non-zeros
302 lines were read
Writing problem data to 'C:\Users\jaime\AppData\Local\Temp\tmps7vil853.glpk.glp'...
253 lines were written
GLPK Simplex Optimizer 5.0
43 rows, 42 columns, 84 non-zeros
Preprocessing...
1 row, 39 columns, 39 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
      0: obj =   1.084900000e+02 inf =   8.188e+01 (1)
     10: obj =   9.473900000e+01 inf =   0.000e+00 (0)
*    25: obj =   1.113400000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:  

In [51]:
solution_rows = []

for i in model.PLAYERS:
    s_2p = value(model.s[i, "2P"])
    s_3p = value(model.s[i, "3P"])
    total_shots_i = s_2p + s_3p
    contrib = value(model.e[i, "2P"]) * s_2p + value(model.e[i, "3P"]) * s_3p
    
    # Only include players who actually shoot
    if total_shots_i > 1e-6:
        solution_rows.append({
            "Player": data.loc[i, "Player"] if "Player" in data.columns else i,
            "2P_shots": s_2p,
            "3P_shots": s_3p,
            "Total_shots": total_shots_i,
            "Expected_points": contrib
        })

solution_df = pd.DataFrame(solution_rows)
solution_df = solution_df.sort_values(by="Expected_points", ascending=False)

print("\nOptimal shot allocation:")
print(solution_df)

total_expected_points = solution_df["Expected_points"].sum()
print(f"\nTotal expected points: {total_expected_points:.2f}")
print(f"Total shots used: {solution_df['Total_shots'].sum():.0f} (should be {S_team})")



Optimal shot allocation:
                    Player  2P_shots  3P_shots  Total_shots  Expected_points
0       Bennedict Mathurin    13.125     6.250       19.375        24.363750
1            Pascal Siakam     8.125     6.375       14.500        14.785375
5            James Wiseman     6.250     3.000        9.250        11.500000
2            Aaron Nesmith     0.000     9.375        9.375        10.490625
4               Obi Toppin     7.875     0.000        7.875         9.954000
7           Isaiah Jackson     7.125     0.000        7.125         7.894500
3          Andrew Nembhard     0.000     7.750        7.750         7.533000
10            Tony Bradley     4.750     0.000        4.750         6.203500
6   Jeremiah Robinson-Earl     3.750     0.000        3.750         5.002500
12             Mac McClung     3.375     0.000        3.375         4.218750
9                 Jay Huff     0.000     4.250        4.250         4.131000
8              RayJ Dennis     2.125     0.000    

## Part c. Sensitivities associated with each constraint

In [52]:
print("\nDual values (shadow prices) for all active constraints:\n")

for c in model.component_objects(Constraint, active=True):
    print(f"Constraint block: {c.name}")
    for index in c:
        constr = c[index]
        dual_val = model.dual.get(constr, 0.0)
        print(f"  {constr.name}: dual = {dual_val:.4f}")
    print()


Dual values (shadow prices) for all active constraints:

Constraint block: TotalShots
  TotalShots: dual = 0.9700

Constraint block: Max2PShots
  Max2PShots[0]: dual = 0.1720
  Max2PShots[1]: dual = 0.0000
  Max2PShots[2]: dual = 0.0000
  Max2PShots[3]: dual = 0.0000
  Max2PShots[4]: dual = 0.0000
  Max2PShots[5]: dual = 0.2940
  Max2PShots[6]: dual = 0.0000
  Max2PShots[7]: dual = 0.0000
  Max2PShots[8]: dual = 0.1500
  Max2PShots[9]: dual = 0.3640
  Max2PShots[10]: dual = 0.1380
  Max2PShots[11]: dual = 0.2120
  Max2PShots[12]: dual = 0.0000
  Max2PShots[13]: dual = 0.0000
  Max2PShots[14]: dual = 0.3360
  Max2PShots[15]: dual = 0.0000
  Max2PShots[16]: dual = 0.0000
  Max2PShots[17]: dual = 0.0000
  Max2PShots[18]: dual = 0.2800
  Max2PShots[19]: dual = 0.0300
  Max2PShots[20]: dual = 0.0000

Constraint block: Max3PShots
  Max3PShots[0]: dual = 0.5300
  Max3PShots[1]: dual = 0.1130
  Max3PShots[2]: dual = 0.1490
  Max3PShots[3]: dual = 0.0000
  Max3PShots[4]: dual = 0.0020
  Max3PS

### Interpretation of Sensitivity Results

In a linear optimization model, each constraint has an associated dual value that measures how the objective would change if the constraint was altered i.e. the effect of the constraints in the optimal objective.

- A non-zero dual value indicates that the corresponding variable is not a 'binding' constraint, which indicates that it has a direct effect on the solution and is 'active' in the model. In our context, allowing one additional shot would increase the team’s expected points by approximately the value of the dual

- Conversely, a zero dual value means the constraint is 'non-binding': the player is not using up their full shooting capacity in the optimal solution, so increasing their shot limit would not change the objective. Zero duals often occur for less efficient shooters, role players, or players whose shot type (2P or 3P) is not used by the optimizer.

In summary, non-zero dual values identify the constraints that impact team’s performance, while zero dual values correspond to constraints that are irrelevant to the model.

#### Total Shots Constraint:

This constraint has a dual value of 0.942, which represents the marginal value of one additional team shot within the current optimal solution. In other words, if the Pacers shot one additional shot, the expected points would increase by approximately 0.942 points.

This value is binding constant: the team is using up all available shot attempts and having more would improve performance.


#### 2-point Shots Constraint:

The dual value associated with each player’s 2-point constraint tells us how valuable it would be to allow that player to take one additional 2-point attempt.

We can see that there are several players with a zero dual value, which are Aaron Nesmith, Jarace Walker, Andrew Nembhard, Ben Sheppard, Jeremiah Robinson-Earl, RayJ Dennis, Monte Morris, Cody Martin, T.J. McConnell, Johnny Furphy and Taelon Peter. These are players to which the optimizer does not assign all their allowed shots because doing so would not increase the expected points. This may be due to the players being less efficient scorers or low-usage offensive players.

The rest of players have positive dual value, meaning that the player is already taking as many 2-pointers as allowed, and giving them one more shot would increase the team's expected points by approximately the dual value.


#### 3-point Shots Constraint:

The interpretation is equivalent to the 2-pointers one.

We can see that only five players would increase the team's expected points by taking more threes. These players are Bennedict Mathurin, Pascal Siakam, Aaron Nesmith, Quenton Jackson and Johnny Furphy.


## Part d. Mixed-Integer Optimization Problem - Optimal Lineup 

### Mathematical Formulation of the Problem

### Sets

  - $P$: set of all players available.

Position subsets:
  - $P^{PG} \subseteq P$ represents the set of point guards (PGs).
  - $P^{SG} \subseteq P$ represents the set of shooting guards (SGs).
  - $P^{F} \subseteq P$ represents the set of forwards (SFs or PFs).
  - $P^{C} \subseteq P$ represents the set of centers (Cs).

### Parameters

For each player $i \in P$:

- $v_{i}$: valuation (performance score) for each player $i$.

We are going to compute this as follows:

- $v_{i} = \mathbf{FG\%}_{i} + \mathbf{FT\%}_{i} + \mathbf{3P\%}_{i} + \mathbf{PTS}_{i} + \mathbf{STL}_{i} + \mathbf{TRB}_{i} + \mathbf{AST}_{i} + \mathbf{BLK}_{i} - \mathbf{TOV}_{i} - \mathbf{PF}_{i}$

We define a linear performance index that rewards scoring efficiency (shooting percentages), points, rebounds, assists, steals and blocks, and penalizes negative aspects of the game as turnovers and personal fouls.

### Decision Variables

- $x_{i} = 
\begin{cases}
    1, & \text{if player } i \text{ is selected in the starting lineup,} \\
    0, & \text{otherwise.}
\end{cases}$

  $x_{i} \in \{0, 1\} \quad \forall i \in P$.


### Objective Function

We want to maximize the total valuation of the selected lineup.

\begin{alignat*}{2}
\max \quad & \sum_{i \in P} v_{i} x_{i} \\
\text{s.t.} \quad & \sum_{i \in P} x_{i} = 5 & \quad & \text{(lineup size)} \\
& \sum_{i \in P^{\text{PG}}} x_{i} \geq 1 & \quad & \text{(at least 1 PG)} \\
& \sum_{i \in P^{\text{SG}}} x_{i} \geq 1 & \quad & \text{(at least 1 SG)} \\
& \sum_{i \in P^{\text{C}}} x_{i} \geq 1 & \quad & \text{(at least 1 C)} \\
& \sum_{i \in P^{\text{F}}} x_{i} \geq 2 & \quad & \text{(at least 2 forwards)} \\
& x_{i} \in \{0, 1\} \quad \forall i \in P & \quad & \text{(binary decisions)}
\end{alignat*}


In [53]:
import pandas as pd
from pyomo.environ import *
import numpy as np
from pyomo.environ import (
    ConcreteModel, Set, Param, Var, Binary,
    Objective, Constraint, maximize, SolverFactory, value
)

In [54]:
data = pd.read_csv("pacers_stats_per_game.csv" ,sep =",", encoding='latin-1')

data = data[data["Player"] != "Team Totals"].reset_index(drop=True)

# columns to convert to numeric
num_cols = [
    "Age", "G", "GS", "MP", "FG", "FGA", "FG%", "3P", "3PA", "3P%",
    "2P", "2PA", "2P%", "eFG%", "FT", "FTA", "FT%", "ORB", "DRB",
    "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS"
]
for c in num_cols:
    data[c] = pd.to_numeric(data[c], errors="coerce")

data["Pos"] = data["Pos"].fillna("").astype(str)

# Columns for valoration metrics

cols_valoration = ["FG%", "FT%", "3P%", "PTS", "STL", "TRB", "AST", "BLK", "TOV", "PF"]

for c in cols_valoration:
    data[c] = pd.to_numeric(data[c], errors="coerce")
    data[c] = data[c].fillna(0.0)

In [55]:

data["valoration"] = (
    data["FG%"] + data["FT%"] + data["3P%"] +
    data["PTS"] + data["STL"] + data["TRB"] +
    data["AST"] + data["BLK"] -
    data["TOV"] - data["PF"]
)
data["valoration"] = data["valoration"].fillna(0.0)
data

,Rk,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player-additional,valoration
0,1.0,Bennedict Mathurin,23.0,SF,2,2,36.5,8.5,15.5,0.548,...,5.5,7.0,2.5,0.0,0.0,2.5,3.0,31.0,mathube01,36.933
1,2.0,Pascal Siakam,31.0,PF,12,12,35.0,8.8,19.3,0.453,...,5.3,7.1,5.1,1.2,0.3,2.4,3.3,24.6,siakapa01,34.070
2,3.0,Aaron Nesmith,26.0,SF,11,11,30.5,4.9,13.4,0.367,...,2.9,4.5,1.5,0.8,0.3,0.7,2.5,15.5,nesmiaa01,20.935
3,4.0,Jarace Walker,22.0,PF,13,8,29.2,3.3,11.3,0.293,...,4.7,5.4,3.2,0.5,0.5,2.2,2.1,10.2,walkeja02,16.874
4,5.0,Andrew Nembhard,26.0,PG,6,6,28.0,5.5,13.8,0.398,...,1.7,1.8,6.5,0.3,0.2,2.0,2.5,18.0,nembhan01,23.990
5,6.0,Obi Toppin,27.0,PF,3,0,27.3,5.0,12.0,0.417,...,5.7,6.7,1.7,1.0,0.0,2.0,2.3,14.0,toppiob01,20.693
6,7.0,Ben Sheppard,24.0,SG,13,5,25.1,2.4,7.5,0.316,...,3.7,4.5,1.6,0.5,0.2,0.7,2.6,6.5,sheppbe01,11.326
7,8.0,Jeremiah Robinson-Earl,25.0,PF,8,3,20.9,2.1,5.6,0.378,...,3.6,6.4,0.9,0.6,0.0,0.8,1.1,5.6,robinje02,13.067
8,9.0,Quenton Jackson,27.0,PG,5,3,20.2,4.0,7.4,0.541,...,2.4,3.4,3.6,1.0,0.2,1.2,2.0,11.8,jacksqu01,18.563
9,10.0,James Wiseman,24.0,C,1,1,20.0,2.0,3.0,0.667,...,0.0,4.0,0.0,0.0,1.0,3.0,2.0,4.0,wisemja01,4.667


In [56]:
players_idx = list(data.index)


PG_idx = [i for i in players_idx if "PG" in data.loc[i, "Pos"]]
SG_idx = [i for i in players_idx if "SG" in data.loc[i, "Pos"]]
C_idx  = [i for i in players_idx if data.loc[i, "Pos"] == "C" or " C" in data.loc[i,"Pos"]]
F_idx  = [i for i in players_idx if any(p in data.loc[i, "Pos"] for p in ["SF", "PF"])]


valoration_dict = data["valoration"].to_dict()

In [57]:
from pyomo.environ import *

import pandas as pd
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, NonNegativeReals, maximize
from itertools import combinations  


model = ConcreteModel()
model.PLAYERS = Set(initialize=players_idx)

model.valoration = Param(
    model.PLAYERS,
    initialize=valoration_dict
)

model.x = Var(model.PLAYERS, domain=Binary)


In [58]:
def objective_rule(m):
    return sum(m.valoration[i] * m.x[i] for i in m.PLAYERS)

model.OBJ = Objective(rule=objective_rule, sense=maximize)
def lineup_size_rule(m):
    return sum(m.x[i] for i in m.PLAYERS) == 5

model.LineupSize = Constraint(rule=lineup_size_rule)

# 5.2. Al menos 1 base (PG)
def pg_rule(m):
    return sum(m.x[i] for i in PG_idx) >= 1

model.MinPG = Constraint(rule=pg_rule)

# 5.3. Al menos 1 escolta (SG)
def sg_rule(m):
    return sum(m.x[i] for i in SG_idx) >= 1

model.MinSG = Constraint(rule=sg_rule)

# 5.4. Al menos 1 pívot (C)
def c_rule(m):
    return sum(m.x[i] for i in C_idx) >= 1

model.MinC = Constraint(rule=c_rule)

# 5.5. Al menos 2 aleros/interiores (SF o PF)
def f_rule(m):
    return sum(m.x[i] for i in F_idx) >= 2

model.MinF = Constraint(rule=f_rule)

# (1) Máximo 2 jugadores "G-League"
#     Elegimos algunos nombres que queremos tratar como G-League o menos asentados
gleague_names = [
    "Mac McClung",
    "Johnny Furphy",
    "Taelon Peter",
    "RayJ Dennis",
    "Ben Sheppard",
    "Quenton Jackson"
]

G_idx = [i for i in players_idx if data.loc[i, "Player"] in gleague_names]

if len(G_idx) > 0:
    model.MaxGLeague = Constraint(
        expr=sum(model.x[i] for i in G_idx) <= 2
    )

# (2) Jugadores incompatibles: por ejemplo, dos pívots tradicionales que no queremos juntos
#     (cambia los nombres si quieres otros)
incompat_pair = ("Isaiah Jackson", "Tony Bradley")
incompat_idx = [i for i in players_idx if data.loc[i, "Player"] in incompat_pair]

if len(incompat_idx) == 2:
    i1, i2 = incompat_idx
    model.Incompatibility = Constraint(expr=model.x[i1] + model.x[i2] <= 1)

# (3) Regla "si juega la estrella, necesito un defensor 3&D"
#     Ejemplo: si juega Pascal Siakam, al menos uno de Cody Martin o Ben Sheppard
star_name = "Pascal Siakam"
defenders = ["Cody Martin", "Ben Sheppard"]

star_idx = [i for i in players_idx if data.loc[i, "Player"] == star_name]
def_idx  = [i for i in players_idx if data.loc[i, "Player"] in defenders]

if len(star_idx) == 1 and len(def_idx) >= 1:
    i_star = star_idx[0]
    # x_star <= sum x_def
    model.StarNeedsDefender = Constraint(
        expr=model.x[i_star] <= sum(model.x[j] for j in def_idx)
    )

In [59]:
solver = SolverFactory("glpk")  # cambia a "cbc", "gurobi", etc. si usas otro
results = solver.solve(model, tee=True)

print("\nEstado del solver:", results.solver.status)
print("Condición de terminación:", results.solver.termination_condition)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\jaime\AppData\Local\Temp\tmpccptacme.glpk.raw --wglp C:\Users\jaime\AppData\Local\Temp\tmpxmmc8n51.glpk.glp
 --cpxlp C:\Users\jaime\AppData\Local\Temp\tmp9lkv3iqs.pyomo.lp
Reading problem data from 'C:\Users\jaime\AppData\Local\Temp\tmp9lkv3iqs.pyomo.lp'...
C:\Users\jaime\AppData\Local\Temp\tmp9lkv3iqs.pyomo.lp:125: warning: lower bound of variable 'x2' redefined
C:\Users\jaime\AppData\Local\Temp\tmp9lkv3iqs.pyomo.lp:125: warning: upper bound of variable 'x2' redefined
8 rows, 20 columns, 51 non-zeros
20 integer variables, all of which are binary
145 lines were read
Writing problem data to 'C:\Users\jaime\AppData\Local\Temp\tmpxmmc8n51.glpk.glp'...
110 lines were written
GLPK Integer Optimizer 5.0
8 rows, 20 columns, 51 non-zeros
20 integer variables, all of which are binary
Preprocessing...
1 hidden covering inequaliti(es) were detected
8 rows, 20 columns, 51 non-zeros
20 integer variables, al

In [60]:
selected_idx = [i for i in model.PLAYERS if value(model.x[i]) > 0.5]

best_lineup = data.loc[selected_idx].copy()
total_val = best_lineup["valoration"].sum()

print("\n=== Mejor quinteto (apartado d, MILP con restricciones lógicas) ===")
print(f"Valoration total = {total_val:.2f}\n")

cols_show = ["Player", "Pos", "PTS", "TRB", "AST", "STL", "BLK", "TOV", "PF", "valoration"]
print(best_lineup[cols_show].to_string(index=False))

print("\nDistribución de posiciones en el quinteto:")
print(best_lineup["Pos"].value_counts())


=== Mejor quinteto (apartado d, MILP con restricciones lógicas) ===
Valoration total = 119.29

            Player Pos  PTS  TRB  AST  STL  BLK  TOV  PF  valoration
Bennedict Mathurin  SF 31.0  7.0  2.5  0.0  0.0  2.5 3.0      36.933
     Pascal Siakam  PF 24.6  7.1  5.1  1.2  0.3  2.4 3.3      34.070
   Andrew Nembhard  PG 18.0  1.8  6.5  0.3  0.2  2.0 2.5      23.990
      Ben Sheppard  SG  6.5  4.5  1.6  0.5  0.2  0.7 2.6      11.326
    Isaiah Jackson   C  8.1  6.1  0.9  0.6  0.3  1.2 3.2      12.975

Distribución de posiciones en el quinteto:
Pos
SF    1
PF    1
PG    1
SG    1
C     1
Name: count, dtype: int64
